# <center> **Compte-rendu projet Optimisation**
<div align = right> Camille Dubois & Florian Garbe

On pose la fonction suivante à minimiser :
$$
\begin{align*}
 f: p= \left(\begin{matrix} p_1 \\ \vdots\\ p_N \end{matrix}\right) \longrightarrow \sum_{i=0}^{N-1} \|p_{i+1}-R_i\Delta p_{i,i+1} -p_i \|_2^2 + \gamma \sum_{i, j \in L}\|p_j - R_i\Delta p_{i,j}-p_i\|_2^2
 \\
 c_1(p) = p_{,3} - \pi \leq 0 \\
 c_2(p) = p_{,3} + \pi \leq 0
 \end{align*}
$$



Ainsi les variables de décisions sont $p = \left(\begin{matrix} p_1 \\ \vdots \\ p_N\end{matrix}\right) \in \mathbb{M}_{3, N} $ au nombre de $3N$.